#Association Rule Using Pycaret

#Installing Pycaret Library

In [1]:
!pip install pycaret==3.0.0.rc1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.1/538.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.6/172.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.1 MB/s eta 0:00

# Loading Dataset

*   Mount drive and load the dataset the dataset using Pandas library.
*   Dataset used for analysis is Kaggle's Market Basket Dataset

In [6]:
from google.colab import drive
drive.mount('/content/drive')

  and should_run_async(code)

  self._read_thread.setDaemon(True)



Mounted at /content/drive


In [18]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/DM-Assignment-Dataset/Lab2_store_data.csv",header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,butter,light mayo,fresh bread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,burgers,frozen vegetables,eggs,french fries,magazines,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7499,escalope,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Building the Model

In [23]:
# Reshape the data to long format
long_format_data = pd.melt(df.reset_index(), id_vars=['index'], value_name='item').drop('variable', axis=1).dropna()
long_format_data.columns = ['transaction_id', 'item_id']

long_format_data.head()

,transaction_id,item_id
0,0,shrimp
1,1,burgers
2,2,chutney
3,3,turkey
4,4,mineral water


In [24]:
from pycaret.arules import *
s = setup(data=long_format_data, transaction_id='transaction_id', item_id='item_id')

Description,Value
session_id,6300
# Transactions,7501
# Items,119
Ignore Items,None


In [25]:
association_Rule_model =create_model(threshold=0.05, metric='support')

# Model Results

In [26]:
association_Rule_model


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(spaghetti),(mineral water),0.1741,0.2384,0.0597,0.3430,1.4391,0.0182,1.1593,0.3694
1,(mineral water),(spaghetti),0.2384,0.1741,0.0597,0.2506,1.4391,0.0182,1.1020,0.4006
2,(chocolate),(mineral water),0.1638,0.2384,0.0527,0.3214,1.3483,0.0136,1.1224,0.3090
3,(mineral water),(chocolate),0.2384,0.1638,0.0527,0.2209,1.3483,0.0136,1.0733,0.3392
4,(eggs),(mineral water),0.1797,0.2384,0.0509,0.2834,1.1888,0.0081,1.0628,0.1936
5,(mineral water),(eggs),0.2384,0.1797,0.0509,0.2136,1.1888,0.0081,1.0432,0.2086
